In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
# Data of each sample
data = dict()

data['HCH3944'] = {
    'sampling_freq': 7,
    'first_lineage': {
        'name': 'BA.5.2*',
        'sig': ['C1627T', 'G12160A', 'T21765-', 'A21766-', 'C21767-', 'A21768-', 'T21769-', 'G21770-', 'T22917G', 'T23018G', 'G26529A', 'C27889T', 'A28330G']
    },
    'second_lineage': {
        'name': 'BA.2.3.7',
        'sig': ['C2156T', 'C8991T', 'C9866T', 'A21851G', 'A23040G', 'G25314T', 'C25810T', 'C26858T', 'A27259C', 'G27382C', 'A27383T', 'T27384C', 'G29239A']
    }
}


data['HCH3987'] = {
    'sampling_freq': 7,
    'first_lineage': {
        'name': 'BA.5.2.1*',
        'sig': ['C1627T', 'G12160A', 'T21765-', 'A21766-', 'C21767-', 'A21768-', 'T21769-', 'G21770-', 'T22917G', 'T23018G', 'G26529A', 'A27038G', 'C27889T', 'A28330G']
    },
    'second_lineage': {
        'name': 'BA.2.3.7',
        'sig': ['C2156T', 'C8991T', 'C9866T', 'A21851G', 'A23040G', 'G25314T', 'C25810T', 'C26858T', 'A27259C', 'G27382C', 'A27383T', 'T27384C', 'G29239A']
    }
}


data['HCH4149'] = {
    'sampling_freq': 7,
    'first_lineage': {
        'name': 'BA.2.75.5*',
        'sig': ['C3796T', 'C3927T', 'C4586T', 'C5183T', 'A12444G', 'G15451A', 'C20235T', 'A22001G', 'T22016C', 'C22033A', 'A22190G',\
                'G22331A', 'G22577C', 'G22599C', 'A22629C', 'G22898A', 'T22942G', 'T23031C', 'C25416T', 'A26275G']
    },
    'second_lineage': {
        'name': 'BA.2.3.7',
        'sig': ['C2156T', 'C8991T', 'G29239A', 'C25810T', 'G25314T', 'A21851G', 'A23040G']
    }
}


data['HCH4333'] = {
    'sampling_freq': 5,
    'first_lineage': {
        'name': 'BA.5.3.1*',
        'sig': ['C1931A', 'T2954C', 'C11750T', 'G12160A', 'T14257C', 'G16935A', 'T21765-', 'A21766-', 'C21767-', 'A21768-', 'T21769-', 'G21770-', 'A22893C', \
                'T22917G', 'T22942A', 'T23018G', 'G26529A', 'C27889T', 'C28312T', 'G28681T']
    },
    'second_lineage': {
        'name': 'BA.2.3.7',
        'sig': ['C2156T', 'C8991T', 'C9866T', 'C26858T', 'T27384C', 'G27382C', 'A27259C', 'C25810T', 'G29239A', 'G25314T', 'A21851G', 'A27383T', 'A23040G']
    }
}

In [ ]:
# ======================  VARIABLES TO BE ASSIGNED  ===================================
patient_id = 'HCH4149'

fig_size = (18, 7.5) # Figure size (19, 6.2) or (18, 7.5) 
subtitle1 = 'A'
subtitle2 = 'B'

# ========= Modify the filenames accordingly ========
working_directory_path = '.'
sample_info_file = 'NHRI_Run9-17_sample information.xlsx'
# modify the list of column names if other information in the info_file to be annotated on the plot
columns_for_info_label = ['Collection date', 'Sequence coverage', 'Pango lineage'] 
total_reads_fname = 'alignedReads.txt'
base_freq_fname = 'base.fqy'
SARS2ref = 'NC_045512.2' # SARS-CoV-2 reference ID
# ========================================================================================

sampling_freq = data[patient_id]['sampling_freq']   # number of tests taken
first_lineage_sig = data[patient_id]['first_lineage']['sig']
second_lineage_sig = data[patient_id]['second_lineage']['sig']

# sample information list
df_sample_info = pd.read_excel(f'{working_directory_path}/data/{sample_info_file}')
# column value formatting, comment out if not needed
df_sample_info['Sequence coverage'] = df_sample_info['Sequence coverage'].apply(lambda x: str(x)+'%')
df_sample_info['Collection date'] = df_sample_info['Collection date'].apply(lambda x: x.strftime("%Y-%m-%d"))


# sorting the signatures by their position
first_lineage_sig = sorted(first_lineage_sig, key=lambda x: int(x[1:-1]))
second_lineage_sig = sorted(second_lineage_sig, key=lambda x: int(x[1:-1]))

# list of all sample No. for this patient
sample_no = [patient_id] + [f'{patient_id}-{str(i)}' for i in range(2,sampling_freq+1)]

In [ ]:
# Build a dictionary to hold data of each sample
dict_samples = dict()

# coordinates of the signatures
coordinates = [int(c[1:-1]) for c in first_lineage_sig + second_lineage_sig]


# Iterate through each samples of this patient
for sample in sample_no: 
    # Sample directory
    sample_directory_path = f'{working_directory_path}/data/{patient_id}/{sample}'

    # Get sample info 
    sample_info = []
    index = df_sample_info.loc[df_sample_info['Sample_No'] == sample].index.item()
    for c in columns_for_info_label:
        sample_info.append(df_sample_info.at[index, c])
    sample_info = ' | '.join(sample_info)

    # Total read counts for this sample library
    f = open(f'{sample_directory_path}/{total_reads_fname}', "r")
    library_total_reads = int(f.read()) 
    f.close()

    # Extract out the signature positions (discard those non-signature positions)
    df = pd.read_csv(f'{sample_directory_path}/{base_freq_fname}', sep='\t')
    df = df[['ref', 'coord', 'depth', 'A', 'T', 'C', 'G', 'N', 'Del']]
    df = df.loc[df['ref'] == SARS2ref]
    df = df.loc[df['coord'].isin(coordinates)]
    df = df.rename(columns={'Del':'-'}).drop(['ref'], axis=1).set_index('coord')
    # if sequencing depth not available for certain signatures, replace with zero   
    if len([x for x in coordinates if not x in list(df.index)]) >0:
        for p in [x for x in coordinates if not x in list(df.index)]:
            df.loc[p] = [0, 0, 0 ,0, 0, 0, 0]

    
    # Append into the dictionary with the sample No. as key
    dict_samples[sample] = [df, library_total_reads, sample_info]
    
print(dict_samples.keys())

In [ ]:
# ALLELE FREQUENCY : read counts of variant/ depth

# Build a dataframe with signatures as column names and sample numbers as row index
df_af = pd.DataFrame(columns = first_lineage_sig + second_lineage_sig, index = list(dict_samples.keys()), dtype=float)

# Calculate the allele frequency of the signature variant
'''
for instance, A21851G(S:K97E) is a signature of BA.2.3.7, 
the allele frequency will be calculated by (count of reads with 'G'/ depth) at position 21851
'''
for sample, data in dict_samples.items():
    for sig in first_lineage_sig + second_lineage_sig:
        coord = int(sig[1:-1])        
        alt_base = sig[-1]
        # Allele frequency
        if data[0].at[coord, 'depth'] == 0:
            sig_af = -1
        else:
            sig_af = data[0].at[coord, alt_base] / data[0].at[coord, 'depth']
        # write the value into dataframe
        df_af.at[sample, sig] = sig_af

df_af

In [ ]:
# SEQUENCING DEPTH in RPM (Reads per million): 
# Coverage at a base position / total number of mapped reads in this sample library

# Build a dataframe with signatures as column names and sample numbers as row index
df_rpm_depth = pd.DataFrame(columns = first_lineage_sig + second_lineage_sig, index = list(dict_samples.keys()), dtype=float)

for sig in first_lineage_sig + second_lineage_sig:
    coord = sig[1:-1]
    df_rpm_depth.rename(columns = {sig: coord}, inplace = True)

# Calculate the RPM at signature base, which reflecting the normalized sequencing depth at this position 
'''
for instance, A21851G(S:K97E) is a signature of BA.2.3.7, 
the depth in RPM will be calculated by (10^6 * count of reads (depth) at 21851 / library total read counts)
'''
for sample, data in dict_samples.items():
    for coord in list(df_rpm_depth.columns):
        # Depth
        sig_rpm_depth = (pow(10, 6) * data[0].at[int(coord), 'depth']) / data[1]
        # write the value into dataframe
        df_rpm_depth.at[sample, coord] = sig_rpm_depth

df_rpm_depth

In [ ]:
# SAMPLE INFO (Metadata) for the right y-axis label
sample_info_list = []
for sample, data in dict_samples.items():    
    sample_info_list.append(data[2])
sample_info_list

In [ ]:
# PLOT: ALLELE FREQUENCY & DEPTH IN RPM

col_n_1stlineage = len(first_lineage_sig)
col_n_2ndlineage = len(second_lineage_sig)

fig, ax = plt.subplots(2, figsize=fig_size) 

# =========== ALLELE FREQUENCY ==========
values_af = df_af.to_numpy(dtype=float)

sns.heatmap(values_af, cmap="RdYlBu_r", vmin=0, vmax=1, center = 0.5, square=False, 
            cbar_kws={"shrink": 0.9, "anchor": (1.5, 0.5)}, ax=ax[0])

ax[0].xaxis.tick_top()
ax[0].xaxis.set_label_position('top')
ax[0].axhline(y = 0, color='k',linewidth = 2)
ax[0].axhline(y = values_af.shape[0], color = 'k', linewidth = 2)  
ax[0].axvline(x = 0, color = 'k', linewidth = 2)  
ax[0].axvline(x = values_af.shape[1], color = 'k', linewidth = 2)
ax[0].axvline(x = col_n_1stlineage, color = 'k', linewidth = 3) # seperating line between two lineages

sns.heatmap(values_af, xticklabels=df_af.columns, yticklabels=df_af.index,
            cmap=plt.get_cmap('binary'), vmin=-2, vmax=0, mask=values_af >= 0, cbar=False, ax=ax[0])

ax[0].collections[0].colorbar.set_label("Allele Frequency (%)", fontsize = 14, labelpad=13)
ax[0].collections[0].colorbar.ax.tick_params(labelsize=13)
ax[0].tick_params(axis='y', length=3, labelrotation=0, pad=8, labelsize = 14)
ax[0].tick_params(axis='x', length=3, labelrotation=90, pad=5, labelsize = 14)

ax_twin = ax[0].twinx()
ax_bbox = ax[0].get_position()
plt.subplots_adjust(bottom=ax_bbox.y0, top=ax_bbox.y1)
ax_twin.set_ylim(ax[0].get_ylim())
ax_twin.set_yticks(ax[0].get_yticks()) 
ax_twin.set_yticklabels(labels=sample_info_list)
ax_twin.tick_params(axis='y', length=0, labelrotation=0, pad=12, labelsize = 14)


# ============ DEPTH IN RPM ============
values_rpm_depth = df_rpm_depth.to_numpy(dtype=float)

# Set limits for color scale according to the values 
max_value = 4 
min_value = -1 
#print(np.log10(values_rpm_depth.max()))
#print(np.log10(np.extract(values_rpm_depth > 0, values_rpm_depth).min()))

# ------ Customed color map ------
def create_color(r, g, b):
    return [r/256, g/256, b/256]
 
def custom_color_palette():
    return LinearSegmentedColormap.from_list("", 
        [
         create_color(215,215,215), #-1, light grey
         create_color(255,255,255), #0, white
         create_color(255,220,100), #1, light yellow        
         create_color(145,220,170), #2, green
         create_color(5,82,5),      #3
         create_color(0,3,0)        #4
        ])
 
cmap = custom_color_palette()
# ---------------------------------

ax[1].xaxis.tick_top()
ax[1].xaxis.set_label_position('top')

sns.heatmap(np.log10(values_rpm_depth, where=values_rpm_depth != 0), 
            cmap=cmap, vmin=min_value, vmax=max_value, square=False, cbar_kws={"shrink": 0.9, "anchor": (1.5, 0.5)}, ax=ax[1]) 

ax[1].axhline(y = 0, color='k',linewidth = 3)
ax[1].axhline(y = values_rpm_depth.shape[0], color = 'k', linewidth = 3)  
ax[1].axvline(x = 0, color = 'k', linewidth = 3)  
ax[1].axvline(x = values_rpm_depth.shape[1], color = 'k', linewidth = 3)
ax[1].axvline(x = col_n_1stlineage, color = 'k', linewidth = 3)

sns.heatmap(values_rpm_depth, xticklabels=df_rpm_depth.columns, yticklabels=df_rpm_depth.index,
            cmap=plt.get_cmap('binary_r'), vmin=-1, vmax=1, mask=values_rpm_depth > 0, cbar=False, ax=ax[1])

ax[1].collections[0].colorbar.set_label("log10(RPM)", fontsize = 14, labelpad=13)
ax[1].collections[0].colorbar.ax.tick_params(labelsize=14)
ax[1].tick_params(axis='y', length=3, labelrotation=0, pad=5, labelsize = 14)
ax[1].tick_params(axis='x', length=3, labelrotation=90, pad=5, labelsize = 14)


# ============ Assign subtitles to subfigures ============
ax[0].set_title(subtitle1, x=-0.11, y=1.16, fontweight = 'bold', fontsize = 23)
ax[1].set_title(subtitle2, x=-0.11, y=1.16, fontweight = 'bold', fontsize = 23)

plt.tight_layout()
plt.subplots_adjust(left=0.09)
#plt.savefig(f'{working_directory_path}/{patient_id}_AF-depthRPM.png', dpi=600)
plt.show()